In [1]:
import pandas as pd
import geopandas as gpd
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np
from shapely.geometry import Point
import folium

### Get transport features from buildings and landmarks
Landmarks include: (mostly) railway stations, (1) Marina, (1) transport terminal and (1) bridge  
Buildings: there are 157, but it's not clear what they are

In [66]:
def create_transport_gdf(transport_fp):
    # read in csv
    transport_df = pd.read_csv(transport_fp)
    # Split into latitude and longitude, and convert these to numeric
    transport_df[['Latitude', 'Longitude']] = transport_df['Geo Point'].str.split(',', 1, expand=True)
    transport_df[['Latitude', 'Longitude']]= transport_df[['Latitude', 'Longitude']].apply(pd.to_numeric)
    geometry = [Point(xy) for xy in zip(transport_df.Longitude, transport_df.Latitude)]
    transport_gdf = gpd.GeoDataFrame(transport_df, geometry=geometry)
    return transport_gdf

def create_other_features_gdf(fp):
    df = pd.read_csv(fp)
    geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf

### Read in new extra transport features

In [ ]:
tram_stops_gdf = create_transport_gdf("../Data/TransportHubs/tram-stops.csv")
metro_stations_gdf = create_transport_gdf("../Data/TransportHubs/metro-stations.csv")
bus_stops_gdf = create_transport_gdf("../Data/TransportHubs/bus-stops.csv")

### Read in other features

In [84]:
buildings = create_other_features_gdf("../Cleaned_data/buildings_clean.csv")
landmarks = create_other_features_gdf("../Cleaned_data/landmarks_clean.csv")
furniture = create_other_features_gdf("../Cleaned_data/street_inf_clean.csv")
memorials = create_other_features_gdf("../Cleaned_data/memorials_clean.csv")

In [83]:
drinking_fountains = furniture[furniture['feature'] == 'Drinking Fountain']
landmarks['theme'].unique()

array(['Transport', 'Mixed Use', 'Place Of Assembly',
       'Leisure/Recreation', 'Place of Worship', 'Health Services',
       'Community Use', 'Retail', 'Education Centre', 'Office',
       'Vacant Land', 'Purpose Built',
       'Specialist Residential Accommodation',
       'Residential Accommodation', 'Industrial', 'Warehouse/Store'],
      dtype=object)

In [2]:
buildings_transport = pd.read_csv("../Cleaned_data/buildings_clean.csv")
buildings_transport = buildings_transport[buildings_transport['building_use']=='Transport']

landmarks_transport = pd.read_csv("../Cleaned_data/landmarks_clean.csv")
landmarks_transport = landmarks_transport[landmarks_transport['theme']=='Transport']

### Read in sensor data

In [4]:
melbourne_sensors = pd.read_csv("../Data/melbourne_locations.csv")
melbourne_sensors.rename(columns={'sensor_description': 'Name'}, inplace = True)

In [58]:
geometry_sensors = [Point(xy) for xy in zip(melbourne_sensors.Longitude, melbourne_sensors.Latitude)]
sensors_gdf = gpd.GeoDataFrame(melbourne_sensors, geometry=geometry_sensors)

### Plot transport feature locations

In [80]:
# Set up figure
f = folium.Figure(width=600, height=600)


melbourne_map = folium.Map(location=[-37.81426902667574, 144.97034862684487], zoom_start=16, control_scale=True, min_zoom = 10).add_to(f)
                                     
# melbourne_map = folium.Map(location=[melbourne_sensors.Latitude.mean(),
#                            melbourne_sensors.Longitude.mean()], zoom_start=12, control_scale=True, min_zoom = 10).add_to(f)

                                     # Plot transport landmarks
for index, location_info in landmarks_transport_gdf.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        popup = location_info['sub_theme'], color = 'red', fill_color="red",
                        fill_opacity=1).add_to(melbourne_map)
    
# Plot transport buildings
for index, location_info in buildings_transport_gdf.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='blue', fill_color="blue", fill_opacity=1, popup = 'transport building').add_to(melbourne_map)    
    
# Plot bus stops
for index, location_info in bus_stops_gdf.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='green', fill_color="green", popup = 'bus stop',fill_opacity=1).add_to(melbourne_map)    
    
# Plot tram stops
for index, location_info in tram_stops_gdf.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='purple', fill_color="purple", popup = 'tram stop',fill_opacity=1).add_to(melbourne_map)    
    
# Plot metro stops
for index, location_info in metro_stations_gdf.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='cyan', fill_color="cyan", popup = 'metro',fill_opacity=1).add_to(melbourne_map)
    
# # Plot street furniture
# for index, location_info in furniture.iterrows():
#     folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=2, fill = True,
#                         color='black', fill_color="black", popup = 'furniture',fill_opacity=1).add_to(melbourne_map)    
    
# # Plot drinking fountains
for index, location_info in drinking_fountains.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=5, fill = True,
                        color='black', fill_color="black", popup = 'fountain',fill_opacity=1).add_to(melbourne_map)    

# Plot landmarks
for index, location_info in landmarks.iterrows():
    folium.CircleMarker([location_info["Latitude"], location_info["Longitude"]], radius=2, fill = True,
                        color='green', fill_color="green", popup = 'landmarks',fill_opacity=1).add_to(melbourne_map)        
    
# Plot sensors
for index, location_info in sensors_gdf.iterrows():
    folium.Marker([location_info["Latitude"], location_info["Longitude"]], radius=5,
                  popup = location_info['Name']).add_to(melbourne_map)         
    
melbourne_map

In [ ]:
# Function which converts the geometry column in the gpd of edges into the required format
def get_coords(frame):
    xy = frame.geometry.xy
    longs = xy[0].tolist()
    lats = xy[1].tolist()
    return [list(z) for z in zip(lats, longs)]

gdf_edges = gpd.read_file("../Data/gdf_edges/gdf_edges.shp")
gdf_edges['coords'] = gdf_edges.apply(get_coords, axis=1)

In [18]:
# Set up figure
f = folium.Figure(width=600, height=600)


melbourne_map = folium.Map(location=[-37.81426902667574, 144.97034862684487], zoom_start=16, control_scale=True, min_zoom = 10).add_to(f)
                                      
    
melbourne_map